In this example a simple SFINCS compound flood model will be made, using the underlying Python functions of HydroMT-SFINCS to build a model.

The model is situated in Northern Italy, where a small selection of topography and bathymetry data has already been made available for you to try the examples.

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import geopandas as gpd
from datetime import datetime

from hydromt_sfincs import SfincsModel
from hydromt_sfincs import utils

This example shows how to build a simple SFINCS model on a regular grid, containing an elevation dep-file, offshore water level forcing and an upstream discharge input forcing.
For making a more advanced model including e.g. spatially varying infiltration and roughness, see the example notebook: build_advanced_subgrid_compound_model_from_script.ipynb

In case you want to adjust this example to build a SFINCS model anywhere else in the world, you will have to add your own datasets to HydroMT's data catalog. For more info on that, see the example notebook: example_datasources.ipynb

Steps followed in this notebook to build your SFINCS model:
<ul> 
<li> 1. Open SfincsModel class, set data library and output folder </li>
<li> 2. Specify characteristics of the wanted grid </li>
<li> 3. Load in wanted elevation datasets </li>
<li> 4. Make mask of active and inactive cells </li>
<li> 5. Update mask with water level and outflow boundary cells</li>
<li> 6. Add spatially varying roughness data</li>
<li> 7. Make subgrid derived tables</li>
<li> 8. Add spatially varying infiltration data</li>
<li> 9. Add water level time-series as forcing</li>
<li> 10. Add an upstream discharge time-series as forcing</li>
<li> 11. Add spatially varying rainfall data</li>
<li> 12. Add weirfile</li>
<li> 13. Add observation points</li>
<li> 14. Show model</li>
<li> 15. Save all files</li>
</ul> 

Let's get started!

### 1. Open SfincsModel class, set data library and output folder:

In [ ]:
# Initialize SfincsModel Python class with the artifact data catalog which contains publically available data for North Italy
sf = SfincsModel(data_libs=["artifact_data"], root="sfincs_compound_advanced")#, mode='r+')

### 2. Specify characteristics of the wanted grid and generate grid:

For more info about how to define a grid see: https://sfincs.readthedocs.io/en/latest/input.html#grid-characteristics

In [ ]:
# Specify an input dictionary with the grid settings x0,y0,dx,dy,nmax,mmax,rotation and epsg code.
inp_dict = {
    "x0": 318650,   
    "y0": 5040000,
    "dx": 50.0,
    "dy": 50.0,
    "nmax": 107,
    "mmax": 250,
    "rotation": 27,
    "epsg": 32633,
}

# create SFINCS model with regular grid and characteristics of the input dictionary:
sf.create_grid(grid_type="regular", **inp_dict)

# the input file is automatically updated, and displayed below:
sf.config

### 3. Load in wanted elevation datasets:

In [ ]:
# We would like to use the size of the generated grid, to only load in the data covering that grid, to reduce time
# the active part of the grid is stord in the SfincsModel.region attribute

sf.region

# you can also plot the region with
# sf.region.plot()

In [ ]:
# In this example we want to combine 2 elevation datasets, merit_hydro as elevation and gebco as bathymetry, in that order. 

# Here these 2 are loaded from the datacatalog and added to one list of elevation datasets (you could add more yourself if wanted, in case made available in your data catalog)
da_dep1 = sf.data_catalog.get_rasterdataset("merit_hydro", variables=["elevtn"], geom=sf.region, buffer=5)

da_dep2 = sf.data_catalog.get_rasterdataset("gebco", variables=["elevtn"], geom=sf.region, buffer=5)

# Provide merge arguments together with xr.DataAraay
da_dep_lst = [{"da":da_dep1, "min_valid":0.01}, {"da":da_dep2}]

#NOTE: from the 2nd elevation dataset (gebco) only elevation data below an elevation of 0 meters is used ("zmax":0)

In [ ]:
# Add depth information to modelgrid based on these chosen datasets
dep = sf.create_dep(da_dep_lst=da_dep_lst)

# Make a plot of the merged topobathy, here colour limits are set between an elevation of -5 to 5 meters
sf.grid.dep.plot(x='xc', y='yc',vmin=-5, vmax=5)

### 4. Make mask of active and inactive cells:

For more info about what the msk-file is see: https://sfincs.readthedocs.io/en/latest/input.html#mask-file

In [ ]:
# Here we generate the mask of active (msk=1) and inactive cells (msk=0), determining what cells on your grid should be used.

# Choosing how to choose you active cells can be based on multiple criteria, here we only specify a minimum elevation of -5 meters
sf.create_mask_active( elv_min=-5, reset_mask=True)

# Make a plot of the mask file
sf.mask.plot(x='xc', y='yc')

# NOTE:
# - The given output of HydroMT says "3 gaps outside valid elevation range < 10 km2"
#   HydroMT does some smart filtering that if small groups of inactive cells are found, surrounded by active cells, these are still included as active, in this case 3 gaps.
#   You can control the size of these gaps to filter by adding ",fill_area = 10 km2)" in create_mask_active()
# - reset_bounds=True means that every time you start clean with defining your msk cells, if on False (default) you build on the prevous time you ran that function

### 5. Update mask with water level and outflow boundary cells - including use of polygons:

In [ ]:
# Loading a shapefile clicked by user:
file_name = "data//compound_example_outflow_boundary_polygon.shp"
gdf_include = sf.data_catalog.get_geodataframe(file_name)

# Example of the same, but how to load an existing ascii .pol file with x&y-coordinates, e.g. coming out of Delft Dashboard, here assumed to be in the CRS of the SFINCS model:
# file_name = "XX.pol"
# gdf_include = utils.polygon2gdf(feats=utils.read_geoms(fn=file_name), crs=sf.crs)

In [ ]:
# In SFINCS you can specify cells where you want to force offshore water levels (msk=2), or outflow boundaries (msk=3)
# For more info about what the msk-file is see: https://sfincs.readthedocs.io/en/latest/input.html#mask-file

# Here we add water level cells along the coastal boundary, for cells up to an elevation of -5 meters
sf.create_mask_bounds(btype="waterlevel", elv_max=-5, reset_bounds=True)

# Here we add outflow cells, only where clicked in shapefile along part of the lateral boundaries
sf.create_mask_bounds(btype="outflow", gdf_include=gdf_include, reset_bounds=True)

# Make a plot of the mask file
sf.mask.plot(x='xc', y='yc')

# NOTE:
# - As you can see now, also msk=2 values have been added along the coastal boundary
# - As you can see now, also msk=3 values have been added along the lateral inland boundaries
#  - reset_bounds=True means that every time you start clean with defining your msk=2 cells, if on False (default) you build on the prevous time you ran that function

### 6. Add spatially varying roughness data:

In [ ]:
# --> this used in making subgrid tables

# TODO: load in global data
# da_manning1 = sf.data_catalog.get_rasterdataset("vito", variables=["cn_avg"], geom=sf.region, buffer=5)
# da_manning_lst = [{"da":da_manning1}]
# da_dep_lst = [{"da":da_dep1}, {"da":da_dep2, "zmax":0, "offset":0}]

#
localtiff = "data//merged_man_m00001_n00000.tif"
# ds_xarray = xr.open_dataset(localtiff)
# ds_xarray
# TODO: do polygon change to river

da_manning1 = sf.data_catalog.get_rasterdataset(localtiff, variables=["manning"])
da_manning_lst = [{"da":da_manning1}]

sf.create_manning_roughness(da_manning_lst = da_manning_lst)#, manning_sea = 0.02)

sf.grid.manning.plot(x='xc', y='yc')#,vmin=0, vmax=0.05)

### 7. Make subgrid derived tables:

Add some specific settings:


In [ ]:
# You can specify multiple settings abou how the subgrid derived tables should be made

# Every single grid cell of the flux grid of the size inp.dx by inp.dy is defined into subgrid pixels (default nr_subgrid_pixels = 20).
# For every subgrid pixel the topobathy data is loaded, ideally this consists also of high-resolution DEM datasets that you specify as user.

# In this example with dx=dy=50m, having nr_subgrid_pixels = 20 means we are loading data onto a 2.5 m subpixel grid
# However, the input data of Gebco and Merit_hydro is way coarser, therefore let's set the ratio to 5 for now: 
nr_subgrid_pixels = 10

In [ ]:
# Every single grid cell of the flux grid of the size inp.dx by inp.dy is defined into subgrid pixels (default is 20, nr_subgrid_pixels = 20).
# For every subgrid pixel the topobathy data is loaded, ideally this consists also of high-resolution DEM datasets that you specify as user.

sf.create_subgrid(da_dep_lst=da_dep_lst, da_manning_lst=da_manning_lst, nr_subgrid_pixels=nr_subgrid_pixels,
                  make_dep_tiles=True, make_manning_tiles=True)

# NOTE: we turned on that the merged topobathy and manning tiles of the different (high-res) datasets are written to tiff-files

# NOTE: if you have a very large domain with 100,000s to millions of cells, and very high-resolution datasets, this step might take minutes to hours!!!
#       But good news; when finished succesfully, you can very quickly run very accurate SFINCS simulations!
#       The whole point of the subgrid functionality of SFINCS is that by derived subgrid tables based on high res elevation data, 
#       you either have more accurate results or run on a coarser grid resolution (= much faster) or both

Now we can see what kind of subgrid-derived variables are created:

In [ ]:
sf.subgrid

### 8. Add spatially varying infiltration data:

In [ ]:
# independent from subgrid files

# TODO: load in global data
da_infiltration1 = sf.data_catalog.get_rasterdataset("gcn250", variables=["cn_avg"], geom=sf.region, buffer=5)
da_infiltration_lst = [{"da":da_infiltration1}]

# TODO: do polygon change to at a city

# make file > only possible with setup_cn_infiltration ?

By now we have made all basic SFINCS spatial layers to make the mskfile, infiltrationfile and subgridfiles, now we're going to add some forcing...

### 9. Add water level time-series as forcing:

In [ ]:
# Change period of model simulation time, specified in yyyymmdd HHMMSS --> simulation time here is 24 hours
sf.set_config("tref", "20100201 000000")
sf.set_config("tstart", "20100201 000000")
sf.set_config("tstop", "20100202 000000")

sf.config

a. specify water level input locations:

For more info about what the bndfile is see: https://sfincs.readthedocs.io/en/latest/input_forcing.html#water-level-points

In [ ]:
# Here we specify at what x&y-locations we have measured/modelled input water level data in the bndfile of SFINCS:

# x&y-locations in same coordinate reference system as the grid:
x = [319526, 329195]
y = [5041108, 5046243]

# add to Geopandas dataframe as needed by HydroMT
pnts = gpd.points_from_xy(x, y)
index = [1, 2]  # NOTE that the index should start at one
bnd = gpd.GeoDataFrame(index=index, geometry=pnts, crs=sf.crs)

# show what has been created:
bnd

b. Make up some time-series:

For more info about what the bzsfile is see: https://sfincs.readthedocs.io/en/latest/input_forcing.html#water-level-time-serie

In [ ]:
# Here we specify at what times we are providing water level input, and afterwards what the values are per input location:

# In this case we will provide 3 values (periods=3) between the start (tstart=20100201 000000) and the end (tstop=20100201 120000) of the simulation:
time = pd.date_range(start=utils.parse_datetime(sf.config["tstart"]), end=utils.parse_datetime(sf.config["tstop"]), periods=3)

# and the actual water levels, in this case for input location 1 a water level rising from 0 to 2 meters and back to 0:
bzs = [[0, 0.25], 
       [0.75, 1.0], 
       [0, 0.25]]

bzspd = pd.DataFrame(index=time, columns=index, data=bzs)
bzspd

In [ ]:
# Actually add it to the SFINCS model class:
sf.create_waterlevel_forcing(df_ts = bzspd, gdf_locs=bnd)

# Plot time-series:
sf.plot_forcing()

### 10. Add an upstream discharge time-series as forcing

a. specify discharge input locations: srcfile

For more info about what the srcfile is see: https://sfincs.readthedocs.io/en/latest/input_forcing.html#discharge-points

b. specify discharge time-series: disfile

For more info about what the disfile is see: https://sfincs.readthedocs.io/en/latest/input_forcing.html#discharge-time-series

In [ ]:
# We follow exactly the same steps as for the water level forcing, but now specify 1 location where we specify discharges in m3/s
x = [321732]
y = [5047136]

# add to Geopandas dataframe as needed by HydroMT
pnts = gpd.points_from_xy(x, y)
index = [1]  # NOTE that the index should start at one
src = gpd.GeoDataFrame(index=index, geometry=pnts, crs=sf.crs)

time = pd.date_range(start=utils.parse_datetime(sf.config["tstart"]), end=utils.parse_datetime(sf.config["tstop"]), periods=3)

# and the actual water levels, in this case for input location 1 a water level rising from 0 to 2 meters and back to 0:
dis = [[2.0], 
       [5.0], 
       [2.0]]

dispd = pd.DataFrame(index=time, columns=index, data=dis)

# NOTE: only now we call the function create_discharge_forcing
sf.create_discharge_forcing(df_ts = dispd, gdf_locs=src)

# Plot time-series:
sf.plot_forcing()

In case you want to add other types of forcing, see the example notebook example_forcing.ipynb for other types.

Or have a look at: https://sfincs.readthedocs.io/en/latest/input_forcing.html

### 11. Add spatially varying rainfall data:

In [ ]:
# TODO: load from ERA5 data?
# Here we load in precipitation data from a netcdf file in the artifact data called era5.nc, that contains the variable 'precip'

df_ts = sf.data_catalog.get_dataframe("era5", variables=["precip"], time_tuple=(sf.config['tstart'], datetime(2010,2,4,0,0)))

df_ts


In [ ]:
# TODO: finish this, not totally sure what more needs to be done to write away this spatially varying rainfall

# sf.create_precip_forcing(df_ts=df_ts)

### 12. Add weirfile:

In [ ]:
# In this example specify a 'line' style shapefile for the location of the weir to be added
file_name = "data//compound_example_weirfile_input.shp"
gdf_structures = sf.data_catalog.get_geodataframe(file_name)

gdf_structures

# NOTE: here the shapefile needs values for dimension 'z' being the elevation of the weir

In [ ]:
sf.create_structures(gdf_structures=gdf_structures, stype= 'weir', overwrite= True)

# NOTE: optional: , dz=3.0) - If provided, for weir structures the z value is calculated from the model elevation (dep) plus dz.

### 13. Add observation points

For more info about what the obsfile is see: https://sfincs.readthedocs.io/en/latest/input.html#observation-points

In [ ]:
# Loading a point shapefile clicked by user:
file_name = "data//compound_example_observation_points.shp"
gdf_obs = sf.data_catalog.get_geodataframe(file_name, crs=sf.crs)

sf.create_observation_points(gdf_obs=gdf_obs, overwrite=True)

sf.geoms

# NOTE: overwrite=True makes HydroMT overwrite already existing observation points

### 14. Show model

In [ ]:
# Use predefined plotting function 'plot_basemap' to show your full SFINCS model setup
sf.plot_basemap()

### 15. Save all files

In [ ]:
sf.write() # write all

In [ ]:
# Show created files in folder:
dir_list = os.listdir(sf.root)
print(dir_list)

In [ ]:
# TODO: read hydromt_data.yml to show all used data

Your basemap and forcing figures are saved in the folder 'figs', GIS files (tiff & geojson) of your model setup in 'gis' and intermediate merged subgrid elevation and manning roughness tiles in 'tiles'.

Now you have made a model, you can progress to the notebook: run_sfincs_model.ipynb